In [ ]:
%%bash
set -e

cd /data/ct-clip-app  # adjust if your working copy is elsewhere

# 1. Git identity
git config user.name "Mustafa Ak"
git config user.email "mustafaxacikgoz@gmail.com"

# 2. Enable LFS (if not already)#]
git lfs install

# 3. (Optional) Track large files you care about, e.g., .pt or model artifacts
git lfs track "*.pt" || true
git add .gitattributes || true

# 4. Configure GitHub remote with the token (non-interactive auth)
git remote remove github || true
git remote add github https://$hf_space_backup@github.com/Mustafa-Acikgoz/ct-clip-app.git

# 5. Ensure main branch name
git branch -M main

# 6. Stage everything
git add -A

# 7. Commit
git commit -m "Sync full project to GitHub" || echo "Nothing to commit"

# 8. Push to GitHub
git push -u github main


In [2]:
# %%
# ==============================================================================
# CELL 1: Environment and Dependencies Setup
# ==============================================================================
import os
import sys

# --- 1. Install System Libraries ---
print("--- Installing system library libgl1-mesa-glx ---")
!sudo apt-get update -qq && sudo apt-get install -y -qq libgl1-mesa-glx

# --- 2. Install Python Packages ---
print("\n--- Installing Python packages ---")
!pip install -q --upgrade pip
!pip install -q torch transformers datasets nibabel monai sentencepiece \
                einops vector-quantize-pytorch ema-pytorch beartype \
                opencv-python-headless protobuf huggingface_hub bitsandbytes pandas

# --- 3. Clone CT-CLIP Repository ---
print("\n--- Cloning GitHub repositories ---")
if not os.path.isdir("CT-CLIP"):
    !git clone https://github.com/ibrahimethemhamamci/CT-CLIP.git -q
    print("Cloned CT-CLIP.")
else:
    print("CT-CLIP repository already exists.")

# --- 4. Install CT-CLIP Subpackages ---
print("\n--- Installing CT-CLIP subpackages in editable mode ---")
!pip install -q -e CT-CLIP/transformer_maskgit
!pip install -q -e CT-CLIP/CT_CLIP

print("\n" + "="*50)
print(" ✅ Cell 1 complete.")
print(" 🔴 IMPORTANT: Please restart the kernel now before running Cell 2.")
print("="*50)

--- Installing system library libgl1-mesa-glx ---
sudo: effective uid is not 0, is /usr/bin/sudo on a file system with the 'nosuid' option set or an NFS file system without root privileges?

--- Installing Python packages ---

--- Cloning GitHub repositories ---
Cloned CT-CLIP.

--- Installing CT-CLIP subpackages in editable mode ---
  DEPRECATION: Legacy editable install of transformer-maskgit==0.0.0 from file:///data/ct-clip-app/CT-CLIP/CT-CLIP/transformer_maskgit (setup.py develop) is deprecated. pip 25.3 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [ ]:
# ==============================================================================
# CELL 2: Data Preparation, Feature Extraction (204 Samples), and Saving
# ==============================================================================
import os
import sys
import torch
import pandas as pd
import numpy as np
import monai.transforms as T
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModel

# --- 0. Secure token access (no hardcoded tokens) ---
HF_TOKEN = os.environ.get("HF_Token")
if not HF_TOKEN:
    raise RuntimeError("HF_TOKEN missing from Secrets. Set it in Space Settings → Secrets.")

# --- 1. Add cloned repos to Python path ---
sys.path.insert(0, os.path.abspath("CT-CLIP/transformer_maskgit"))
sys.path.insert(0, os.path.abspath("CT-CLIP"))
from transformer_maskgit.MaskGITTransformer import CTViT
from ct_clip.ct_clip import CTCLIP

# --- 2. Configuration ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"--- Using device: {DEVICE} ---")

# --- 3. Load Pre-trained CT-CLIP Model ---
print("\n--- Loading CT-CLIP Model ---")
text_encoder_name = "microsoft/BiomedVLP-CXR-BERT-specialized"
text_tokenizer_clip = AutoTokenizer.from_pretrained(text_encoder_name, trust_remote_code=True)
text_encoder = AutoModel.from_pretrained(text_encoder_name, trust_remote_code=True)
image_encoder = CTViT(dim=512, codebook_size=8192, image_size=480, patch_size=20,
                     temporal_patch_size=10, spatial_depth=4, temporal_depth=4,
                     dim_head=32, heads=8)
vision_model = CTCLIP(image_encoder=image_encoder, text_encoder=text_encoder,
                      dim_image=294912, dim_text=768, dim_latent=512).to(DEVICE)

print("Downloading CT-CLIP v2 weights...")
ckpt_path = hf_hub_download(
    repo_id="ibrahimhamamci/CT-RATE",
    repo_type="dataset",
    filename="models/CT-CLIP-Related/CT-CLIP_v2.pt",
    token=HF_TOKEN,
)
state = torch.load(ckpt_path, map_location="cpu")
vision_model.load_state_dict(state.get("state_dict", state), strict=False)
vision_model.eval()
for param in vision_model.parameters():
    param.requires_grad = False
print(" ✅ CT-CLIP model loaded and frozen.")

# --- 4. Prepare Labels and Data ---
print("\n--- Preparing labels ---")
labels_path = hf_hub_download(
    repo_id="ibrahimhamamci/CT-RATE",
    repo_type="dataset",
    filename="dataset/multi_abnormality_labels/train_predicted_labels.csv",
    token=HF_TOKEN,
)
df_labels = pd.read_csv(labels_path)
CLASS_NAMES = df_labels.columns[1:].tolist()
print(f"Found {len(CLASS_NAMES)} classes.")

# --- 5. Sample filenames (balanced 204) ---
# === Replaced Sample Filenames ===
sample_filenames = [
    "train_18373_a_2.nii.gz",
    "train_19287_a_1.nii.gz",
    "train_3282_a_1.nii.gz",
    "train_54_a_2.nii.gz",
    "train_12769_a_2.nii.gz",
    "train_7066_a_1.nii.gz",
    "train_2610_a_2.nii.gz",
    "train_9742_b_2.nii.gz",
    "train_11431_a_1.nii.gz",
    "train_7613_b_1.nii.gz",
    "train_644_d_1.nii.gz",
    "train_13314_a_2.nii.gz",
    "train_8278_b_2.nii.gz",
    "train_1117_b_1.nii.gz",
    "train_14817_a_1.nii.gz",
    "train_7519_a_1.nii.gz",
    "train_7885_a_1.nii.gz",
    "train_8907_b_2.nii.gz",
    "train_10455_a_2.nii.gz",
    "train_7392_a_2.nii.gz",
    "train_19972_c_2.nii.gz",
    "train_16252_a_1.nii.gz",
    "train_18102_a_1.nii.gz",
    "train_6022_a_2.nii.gz",
    "train_15635_a_2.nii.gz",
    "train_6160_c_2.nii.gz",
    "train_19885_d_2.nii.gz",
    "train_14325_a_2.nii.gz",
    "train_16861_b_1.nii.gz",
    "train_4514_a_1.nii.gz",
    "train_5373_a_2.nii.gz",
    "train_19541_b_2.nii.gz",
    "train_7730_a_2.nii.gz",
    "train_19804_a_2.nii.gz",
    "train_9199_a_1.nii.gz",
    "train_16924_j_2.nii.gz",
    "train_17830_a_1.nii.gz",
    "train_4942_a_3.nii.gz",
    "train_15885_c_1.nii.gz",
    "train_19132_b_1.nii.gz",
    "train_691_a_2.nii.gz",
    "train_2756_a_2.nii.gz",
    "train_8730_a_1.nii.gz",
    "train_4730_a_1.nii.gz",
    "train_17207_a_1.nii.gz",
    "train_12678_c_2.nii.gz",
    "train_14851_a_2.nii.gz",
    "train_13726_a_1.nii.gz",
    "train_6426_a_2.nii.gz",
    "train_12201_a_5.nii.gz",
    "train_11418_c_2.nii.gz",
    "train_10043_b_2.nii.gz",
    "train_2928_a_2.nii.gz",
    "train_13751_a_1.nii.gz",
    "train_15353_b_1.nii.gz",
    "train_13041_a_1.nii.gz",
    "train_16269_a_2.nii.gz",
    "train_11081_a_1.nii.gz",
    "train_16785_a_2.nii.gz",
    "train_1750_a_1.nii.gz",
    "train_2184_a_1.nii.gz",
    "train_6223_a_1.nii.gz",
    "train_14355_a_2.nii.gz",
    "train_12009_b_1.nii.gz",
    "train_3633_a_1.nii.gz",
    "train_4348_a_1.nii.gz",
    "train_6007_a_1.nii.gz",
    "train_11588_a_1.nii.gz",
    "train_7866_a_2.nii.gz",
    "train_7188_a_1.nii.gz",
    "train_17209_a_2.nii.gz",
    "train_5493_a_2.nii.gz",
    "train_19456_a_2.nii.gz",
    "train_13865_a_1.nii.gz",
    "train_2718_b_1.nii.gz",
    "train_15034_a_2.nii.gz",
    "train_18068_b_1.nii.gz",
    "train_13201_a_2.nii.gz",
    "train_15876_a_1.nii.gz",
    "train_1585_b_1.nii.gz",
    "train_17886_a_2.nii.gz",
    "train_19144_c_2.nii.gz",
    "train_15649_a_2.nii.gz",
    "train_15339_a_1.nii.gz",
    "train_3881_a_1.nii.gz",
    "train_14835_a_1.nii.gz",
    "train_18563_a_2.nii.gz",
    "train_17418_a_2.nii.gz",
    "train_14512_a_1.nii.gz",
    "train_16950_b_1.nii.gz",
    "train_15940_b_1.nii.gz",
    "train_7710_a_2.nii.gz",
    "train_5922_a_2.nii.gz",
    "train_15900_a_1.nii.gz",
    "train_14417_a_1.nii.gz",
    "train_5514_a_2.nii.gz",
    "train_10744_g_1.nii.gz",
    "train_1709_b_2.nii.gz",
    "train_3490_a_2.nii.gz",
    "train_1270_a_1.nii.gz",
    "train_14514_b_2.nii.gz",
    "train_11270_a_1.nii.gz",
    "train_17564_a_2.nii.gz",
    "train_3517_a_1.nii.gz",
    "train_11380_a_1.nii.gz",
    "train_9064_a_1.nii.gz",
    "train_5220_a_1.nii.gz",
    "train_10576_b_2.nii.gz",
    "train_2158_a_2.nii.gz",
    "train_2039_d_2.nii.gz",
    "train_14382_a_2.nii.gz",
    "train_13737_a_2.nii.gz",
    "train_13582_a_2.nii.gz",
    "train_18642_a_2.nii.gz",
    "train_5049_a_2.nii.gz",
    "train_19893_a_1.nii.gz",
    "train_16634_a_1.nii.gz",
    "train_18373_i_1.nii.gz",
    "train_11480_c_1.nii.gz",
    "train_13739_a_4.nii.gz",
    "train_6052_a_1.nii.gz",
    "train_15563_a_2.nii.gz",
    "train_14441_b_1.nii.gz",
    "train_11753_a_1.nii.gz",
    "train_17752_a_2.nii.gz",
    "train_19907_a_2.nii.gz",
    "train_11562_a_1.nii.gz",
    "train_11759_a_1.nii.gz",
    "train_9286_a_1.nii.gz",
    "train_3664_a_2.nii.gz",
    "train_14727_a_2.nii.gz",
    "train_11494_a_1.nii.gz",
    "train_8103_a_2.nii.gz",
    "train_3080_a_2.nii.gz",
    "train_8980_a_1.nii.gz",
    "train_15545_a_2.nii.gz",
    "train_10413_a_1.nii.gz",
    "train_14984_a_2.nii.gz",
    "train_7314_a_1.nii.gz",
    "train_6679_a_2.nii.gz",
    "train_3558_a_2.nii.gz",
    "train_19260_a_2.nii.gz",
    "train_9263_a_1.nii.gz",
    "train_6135_a_1.nii.gz",
    "train_9028_a_1.nii.gz",
    "train_14447_a_1.nii.gz",
    "train_14253_a_1.nii.gz",
    "train_16753_a_2.nii.gz",
    "train_9760_a_1.nii.gz",
    "train_12898_b_1.nii.gz",
    "train_15973_a_2.nii.gz",
    "train_7122_a_2.nii.gz",
    "train_16374_a_2.nii.gz",
    "train_17089_a_2.nii.gz",
    "train_16206_a_2.nii.gz",
    "train_17507_b_1.nii.gz",
    "train_6846_a_2.nii.gz",
    "train_366_a_1.nii.gz",
    "train_2223_a_2.nii.gz",
    "train_11466_a_1.nii.gz",
    "train_4242_a_1.nii.gz",
    "train_1906_a_2.nii.gz",
    "train_15971_a_2.nii.gz",
    "train_7429_a_2.nii.gz",
    "train_5573_a_1.nii.gz",
    "train_9534_b_2.nii.gz",
    "train_13203_a_1.nii.gz",
    "train_10442_a_1.nii.gz",
    "train_10385_a_2.nii.gz",
    "train_12856_a_1.nii.gz",
    "train_10785_a_1.nii.gz",
    "train_12029_a_1.nii.gz",
    "train_494_a_1.nii.gz",
    "train_1292_b_2.nii.gz",
    "train_794_a_1.nii.gz",
    "train_7934_a_1.nii.gz",
    "train_3851_a_2.nii.gz",
    "train_4174_a_2.nii.gz",
    "train_11280_a_2.nii.gz",
    "train_11439_a_2.nii.gz",
    "train_11158_a_1.nii.gz",
    "train_11175_a_1.nii.gz",
    "train_8356_b_1.nii.gz",
    "train_19657_c_2.nii.gz",
    "train_18350_a_1.nii.gz",
    "train_10462_a_2.nii.gz",
    "train_10635_a_1.nii.gz",
    "train_19840_a_2.nii.gz",
    "train_4196_b_1.nii.gz",
    "train_4272_f_1.nii.gz",
    "train_5105_e_1.nii.gz",
    "train_11856_a_1.nii.gz",
    "train_19427_c_1.nii.gz",
    "train_13527_a_1.nii.gz",
    "train_18097_a_2.nii.gz",
    "train_15026_a_2.nii.gz",
    "train_870_a_2.nii.gz",
    "train_7594_a_2.nii.gz",
    "train_1946_a_1.nii.gz",
    "train_11336_b_2.nii.gz",
    "train_16743_a_1.nii.gz",
    "train_4671_a_2.nii.gz",
    "train_9384_a_2.nii.gz",
    "train_1029_a_2.nii.gz",
    "train_18520_a_1.nii.gz",
    "train_10051_a_2.nii.gz",
    "train_12821_a_2.nii.gz",
    "train_1870_a_2.nii.gz",
    "train_4836_c_1.nii.gz",
    "train_9634_a_1.nii.gz",
    "train_16222_a_2.nii.gz",
    "train_16609_a_2.nii.gz",
    "train_10827_a_2.nii.gz",
    "train_18449_a_1.nii.gz",
    "train_8125_e_1.nii.gz",
    "train_4861_a_2.nii.gz",
    "train_16204_a_2.nii.gz",
    "train_2880_a_1.nii.gz",
    "train_2879_a_2.nii.gz",
    "train_464_a_1.nii.gz",
    "train_7758_b_2.nii.gz",
    "train_5227_a_1.nii.gz",
    "train_599_a_1.nii.gz",
    "train_16647_a_1.nii.gz",
    "train_9475_a_2.nii.gz",
    "train_6496_a_1.nii.gz",
    "train_9160_a_1.nii.gz",
    "train_19052_a_1.nii.gz",
    "train_14662_a_2.nii.gz",
    "train_17046_a_1.nii.gz",
    "train_3732_b_2.nii.gz",
    "train_1854_a_2.nii.gz",
    "train_15391_a_2.nii.gz",
    "train_5192_a_2.nii.gz",
    "train_12614_a_1.nii.gz",
    "train_1878_a_1.nii.gz",
    "train_16323_a_1.nii.gz",
    "train_12304_a_2.nii.gz",
    "train_8148_a_2.nii.gz",
    "train_14300_a_1.nii.gz",
    "train_13033_a_1.nii.gz",
    "train_1683_a_1.nii.gz",
    "train_13348_a_2.nii.gz",
    "train_10834_a_1.nii.gz",
    "train_15872_a_1.nii.gz",
    "train_18704_a_2.nii.gz",
    "train_419_a_2.nii.gz",
    "train_10775_a_1.nii.gz",
    "train_19246_a_1.nii.gz",
    "train_1429_b_1.nii.gz",
    "train_4300_a_2.nii.gz",
    "train_14438_a_1.nii.gz",
    "train_18706_a_2.nii.gz",
    "train_3634_a_2.nii.gz",
    "train_15772_a_1.nii.gz",
    "train_10011_a_1.nii.gz",
    "train_11977_a_2.nii.gz",
    "train_13697_a_2.nii.gz",
    "train_3083_a_1.nii.gz",
    "train_1897_d_2.nii.gz",
    "train_4944_a_2.nii.gz",
    "train_6634_a_1.nii.gz",
    "train_1162_a_2.nii.gz",
    "train_10987_a_1.nii.gz",
    "train_12378_b_2.nii.gz",
    "train_15294_a_1.nii.gz",
    "train_9307_a_1.nii.gz",
    "train_19565_a_1.nii.gz",
    "train_2282_a_1.nii.gz",
    "train_3190_a_1.nii.gz",
    "train_11611_a_1.nii.gz",
    "train_19687_c_1.nii.gz",
    "train_6482_a_2.nii.gz",
    "train_11577_b_2.nii.gz",
    "train_18467_a_1.nii.gz",
    "train_7186_a_1.nii.gz",
    "train_13863_a_2.nii.gz",
    "train_12252_a_1.nii.gz",
    "train_6622_a_2.nii.gz",
    "train_17138_a_1.nii.gz",
    "train_1104_a_2.nii.gz",
    "train_7754_d_1.nii.gz",
    "train_16723_a_1.nii.gz",
    "train_19020_a_1.nii.gz",
    "train_11880_d_2.nii.gz",
    "train_13918_a_1.nii.gz",
    "train_1713_d_1.nii.gz",
    "train_14471_a_1.nii.gz",
    "train_9006_a_1.nii.gz",
    "train_15553_a_1.nii.gz",
    "train_11778_a_1.nii.gz",
    "train_3901_a_1.nii.gz",
    "train_594_a_1.nii.gz",
    "train_19704_d_2.nii.gz",
    "train_4494_a_1.nii.gz",
    "train_15596_a_2.nii.gz",
    "train_15080_a_1.nii.gz",
    "train_19969_a_1.nii.gz",
    "train_7988_a_1.nii.gz",
    "train_15283_a_2.nii.gz",
    "train_3746_a_2.nii.gz",
    "train_19368_a_1.nii.gz",
    "train_3963_f_1.nii.gz",
    "train_11683_b_2.nii.gz",
    "train_14466_a_2.nii.gz",
    "train_7376_a_2.nii.gz",
    "train_4352_a_2.nii.gz",
    "train_1922_a_1.nii.gz",
    "train_6967_a_1.nii.gz",
    "train_13308_a_1.nii.gz",
    "train_18977_a_2.nii.gz",
    "train_1526_a_1.nii.gz",
    "train_1272_a_1.nii.gz",
    "train_18463_a_2.nii.gz",
    "train_3843_a_1.nii.gz",
    "train_16551_a_1.nii.gz",
    "train_16719_a_2.nii.gz",
    "train_2212_a_2.nii.gz",
    "train_9822_a_2.nii.gz",
    "train_3990_a_1.nii.gz",
    "train_4791_a_2.nii.gz",
    "train_2901_c_1.nii.gz",
    "train_19860_b_2.nii.gz",
    "train_7835_a_1.nii.gz",
    "train_12518_a_2.nii.gz",
    "train_199_a_2.nii.gz",
    "train_12776_a_1.nii.gz",
    "train_16798_b_2.nii.gz",
    "train_14823_b_2.nii.gz",
    "train_8043_a_2.nii.gz",
    "train_9414_b_1.nii.gz",
    "train_6857_a_2.nii.gz",
    "train_1381_d_2.nii.gz",
    "train_12218_a_2.nii.gz",
    "train_7259_b_1.nii.gz",
    "train_16188_b_2.nii.gz",
    "train_19346_a_2.nii.gz",
    "train_4883_a_2.nii.gz",
    "train_8237_b_1.nii.gz",
    "train_2123_a_1.nii.gz",
    "train_16437_a_1.nii.gz",
    "train_8168_c_1.nii.gz",
    "train_9835_a_2.nii.gz",
    "train_7951_a_1.nii.gz",
    "train_13202_b_1.nii.gz",
    "train_11593_b_2.nii.gz",
    "train_13873_a_2.nii.gz",
    "train_12272_a_1.nii.gz",
    "train_11969_a_2.nii.gz",
    "train_4233_a_1.nii.gz",
    "train_1896_b_2.nii.gz",
    "train_14993_a_1.nii.gz",
    "train_3392_a_2.nii.gz",
    "train_6438_a_1.nii.gz",
    "train_19151_b_1.nii.gz",
    "train_16665_a_1.nii.gz",
    "train_10646_a_2.nii.gz",
    "train_15588_a_1.nii.gz",
    "train_2976_a_2.nii.gz",
    "train_4836_b_1.nii.gz",
    "train_16862_a_1.nii.gz",
    "train_19233_a_2.nii.gz",
    "train_15676_a_1.nii.gz",
    "train_17417_a_2.nii.gz",
    "train_17491_a_1.nii.gz",
    "train_18819_a_1.nii.gz",
    "train_2379_a_2.nii.gz",
    "train_14800_a_1.nii.gz",
    "train_1416_a_1.nii.gz",
    "train_17679_a_1.nii.gz",
    "train_10626_a_2.nii.gz",
    "train_16220_b_1.nii.gz",
    "train_10084_a_2.nii.gz",
    "train_14244_a_1.nii.gz",
    "train_19717_a_2.nii.gz",
    "train_14480_a_2.nii.gz",
    "train_15350_a_1.nii.gz",
    "train_1468_b_2.nii.gz",
    "train_8538_a_1.nii.gz",
    "train_16925_a_2.nii.gz",
    "train_17652_a_2.nii.gz",
    "train_5509_a_1.nii.gz",
    "train_16436_a_2.nii.gz",
    "train_8567_c_2.nii.gz",
    "train_7063_a_2.nii.gz",
    "train_18393_a_2.nii.gz",
    "train_13527_b_2.nii.gz",
    "train_3053_a_1.nii.gz",
    "train_17822_a_1.nii.gz",
    "train_8200_a_1.nii.gz",
    "train_8740_b_1.nii.gz",
    "train_17903_a_2.nii.gz",
    "train_905_a_2.nii.gz",
    "train_14520_a_1.nii.gz",
    "train_17645_a_2.nii.gz",
    "train_9314_b_2.nii.gz",
    "train_1748_a_2.nii.gz",
    "train_11319_a_1.nii.gz",
    "train_13988_a_1.nii.gz",
    "train_18032_a_2.nii.gz",
    "train_5634_e_2.nii.gz",
    "train_2448_a_2.nii.gz",
    "train_19793_a_2.nii.gz",
    "train_1896_b_1.nii.gz",
    "train_5381_a_2.nii.gz",
    "train_1017_a_2.nii.gz",
    "train_12329_a_2.nii.gz",
    "train_12870_a_1.nii.gz",
    "train_12877_b_2.nii.gz",
    "train_7946_a_1.nii.gz",
    "train_11792_a_2.nii.gz",
    "train_6128_a_2.nii.gz",
    "train_6367_a_2.nii.gz",
    "train_7428_a_2.nii.gz",
    "train_8306_a_2.nii.gz",
    "train_17197_c_2.nii.gz",
    "train_14428_a_2.nii.gz",
    "train_17245_a_1.nii.gz",
    "train_12362_a_1.nii.gz",
    "train_15106_a_2.nii.gz",
    "train_17988_a_1.nii.gz",
    "train_8841_a_1.nii.gz",
    "train_3233_b_1.nii.gz",
    "train_5468_a_1.nii.gz",
    "train_8957_a_1.nii.gz",
    "train_19657_a_2.nii.gz",
    "train_7666_a_2.nii.gz",
    "train_19687_a_1.nii.gz",
    "train_15554_a_1.nii.gz",
    "train_2926_a_1.nii.gz",
    "train_8199_a_2.nii.gz",
    "train_17488_a_2.nii.gz",
    "train_1041_b_2.nii.gz",
    "train_16624_a_2.nii.gz",
    "train_11113_a_1.nii.gz",
    "train_2077_a_2.nii.gz",
    "train_17872_a_2.nii.gz",
    "train_2407_a_1.nii.gz",
    "train_1995_a_1.nii.gz",
    "train_17649_a_1.nii.gz",
    "train_12545_a_2.nii.gz",
    "train_9166_a_1.nii.gz",
    "train_19330_a_1.nii.gz",
    "train_19952_a_1.nii.gz",
    "train_10897_a_2.nii.gz",
    "train_8617_a_1.nii.gz",
    "train_3587_a_1.nii.gz",
    "train_150_a_1.nii.gz",
    "train_2565_a_2.nii.gz",
    "train_17714_a_2.nii.gz",
    "train_5469_b_2.nii.gz",
    "train_10979_a_1.nii.gz",
    "train_16296_a_1.nii.gz",
    "train_7252_a_1.nii.gz",
    "train_2826_a_1.nii.gz",
    "train_11401_a_6.nii.gz",
    "train_13751_a_2.nii.gz",
    "train_5860_a_1.nii.gz",
    "train_15288_a_2.nii.gz",
    "train_5367_a_2.nii.gz",
    "train_14154_d_3.nii.gz",
    "train_17079_b_2.nii.gz",
    "train_7776_a_1.nii.gz",
    "train_8334_a_1.nii.gz",
    "train_19154_a_2.nii.gz",
    "train_8953_a_2.nii.gz",
    "train_2057_a_1.nii.gz",
    "train_13024_c_1.nii.gz",
    "train_12550_a_1.nii.gz",
    "train_11597_a_1.nii.gz",
    "train_14927_a_1.nii.gz",
    "train_3348_a_1.nii.gz",
    "train_16761_a_1.nii.gz",
    "train_5257_a_2.nii.gz",
    "train_18132_a_1.nii.gz",
    "train_15200_a_1.nii.gz",
    "train_942_a_1.nii.gz",
    "train_13896_b_2.nii.gz",
    "train_17201_a_1.nii.gz",
    "train_3047_a_1.nii.gz",
    "train_9479_a_1.nii.gz",
    "train_15410_a_1.nii.gz",
    "train_8391_a_1.nii.gz",
    "train_10449_a_1.nii.gz",
    "train_13648_b_2.nii.gz",
    "train_3064_a_1.nii.gz",
    "train_12766_a_1.nii.gz",
    "train_2573_a_2.nii.gz",
    "train_2149_a_2.nii.gz",
    "train_3484_a_1.nii.gz",
    "train_12588_b_2.nii.gz",
    "train_9858_a_2.nii.gz",
    "train_15827_a_2.nii.gz",
    "train_5150_a_1.nii.gz",
    "train_359_c_1.nii.gz",
    "train_19838_a_2.nii.gz",
    "train_18689_a_2.nii.gz",
    "train_13429_a_1.nii.gz",
    "train_2186_a_1.nii.gz",
    "train_8415_a_1.nii.gz",
    "train_14443_b_2.nii.gz",
    "train_4510_a_1.nii.gz",
    "train_6165_b_2.nii.gz",
    "train_10389_a_2.nii.gz",
    "train_10507_a_1.nii.gz",
    "train_12927_a_1.nii.gz",
    "train_7045_a_1.nii.gz",
    "train_9377_b_2.nii.gz",
    "train_19888_a_2.nii.gz",
    "train_15285_c_1.nii.gz",
    "train_14993_d_1.nii.gz",
    "train_2465_a_2.nii.gz",
    "train_5379_a_2.nii.gz",
    "train_3900_a_2.nii.gz",
    "train_9226_a_2.nii.gz",
    "train_16677_a_2.nii.gz",
    "train_8065_d_1.nii.gz",
    "train_8169_b_2.nii.gz",
    "train_11352_a_1.nii.gz",
    "train_16937_a_2.nii.gz",
    "train_12243_a_1.nii.gz",
    "train_16124_a_1.nii.gz",
    "train_19428_b_2.nii.gz",
    "train_16309_a_1.nii.gz",
    "train_8643_a_2.nii.gz",
    "train_112_a_2.nii.gz",
    "train_4172_a_2.nii.gz",
    "train_15646_a_2.nii.gz",
    "train_12463_a_1.nii.gz",
    "train_1786_e_2.nii.gz",
    "train_1636_a_1.nii.gz",
    "train_14698_c_2.nii.gz",
    "train_18655_a_1.nii.gz",
    "train_12365_a_1.nii.gz",
    "train_19518_a_2.nii.gz",
    "train_8550_b_2.nii.gz",
    "train_4111_a_2.nii.gz",
    "train_6377_e_2.nii.gz",
    "train_15764_a_1.nii.gz",
    "train_18751_a_1.nii.gz",
    "train_182_b_2.nii.gz",
    "train_1276_a_1.nii.gz",
    "train_12375_b_1.nii.gz",
    "train_2232_a_1.nii.gz",
    "train_16550_b_1.nii.gz",
    "train_17944_d_2.nii.gz",
    "train_11908_b_1.nii.gz",
    "train_17747_c_1.nii.gz",
    "train_12791_a_2.nii.gz",
    "train_17736_a_2.nii.gz",
    "train_13795_a_1.nii.gz",
    "train_16676_a_1.nii.gz",
    "train_8987_g_1.nii.gz",
    "train_3684_a_2.nii.gz",
    "train_13562_b_1.nii.gz",
    "train_15013_b_1.nii.gz",
    "train_4699_a_2.nii.gz",
    "train_15245_a_1.nii.gz",
    "train_4636_a_1.nii.gz",
    "train_10384_a_2.nii.gz",
    "train_8062_a_2.nii.gz",
    "train_5385_a_1.nii.gz",
    "train_7108_a_2.nii.gz",
    "train_10147_a_1.nii.gz",
    "train_872_a_2.nii.gz",
    "train_16724_b_2.nii.gz",
    "train_13903_b_2.nii.gz",
    "train_7064_a_1.nii.gz",
    "train_6836_a_2.nii.gz",
    "train_16196_a_2.nii.gz",
    "train_14895_a_2.nii.gz",
    "train_5347_a_2.nii.gz",
    "train_4634_a_1.nii.gz",
    "train_10983_b_1.nii.gz",
    "train_3168_a_2.nii.gz",
    "train_954_c_2.nii.gz",
    "train_9167_a_1.nii.gz",
    "train_17883_a_1.nii.gz",
    "train_3310_a_2.nii.gz",
    "train_11324_a_1.nii.gz",
    "train_13606_a_2.nii.gz",
    "train_7474_a_2.nii.gz",
    "train_5928_a_1.nii.gz",
    "train_4632_a_2.nii.gz",
    "train_17610_a_1.nii.gz",
    "train_11608_a_5.nii.gz",
    "train_3899_a_2.nii.gz",
    "train_17927_a_2.nii.gz",
    "train_19920_a_2.nii.gz",
    "train_10089_a_2.nii.gz",
    "train_5275_a_2.nii.gz",
    "train_8748_a_1.nii.gz",
    "train_19505_a_2.nii.gz",
    "train_7735_a_2.nii.gz",
    "train_5567_a_1.nii.gz",
    "train_12094_a_1.nii.gz",
    "train_7701_a_2.nii.gz",
    "train_10094_b_1.nii.gz",
    "train_8031_a_1.nii.gz",
    "train_17470_a_2.nii.gz",
    "train_16566_c_2.nii.gz",
    "train_16972_a_1.nii.gz",
    "train_3712_a_2.nii.gz",
    "train_17867_a_1.nii.gz",
    "train_6211_a_2.nii.gz",
    "train_16834_a_2.nii.gz",
    "train_3176_a_2.nii.gz",
    "train_15286_a_1.nii.gz",
    "train_9899_a_2.nii.gz",
    "train_8168_a_2.nii.gz",
    "train_13874_a_2.nii.gz",
    "train_848_h_1.nii.gz",
    "train_19526_a_2.nii.gz",
    "train_1162_a_1.nii.gz",
    "train_12201_b_2.nii.gz",
    "train_11863_a_1.nii.gz",
    "train_7214_e_2.nii.gz",
    "train_3821_a_5.nii.gz",
    "train_19669_a_2.nii.gz",
    "train_3771_a_1.nii.gz",
    "train_17097_a_4.nii.gz",
    "train_1948_a_1.nii.gz",
    "train_5238_a_1.nii.gz",
    "train_3251_a_1.nii.gz",
    "train_2364_a_2.nii.gz",
    "train_6791_a_1.nii.gz",
    "train_1132_a_2.nii.gz",
    "train_2658_a_2.nii.gz",
    "train_3863_b_1.nii.gz",
    "train_12402_a_1.nii.gz",
    "train_7006_b_2.nii.gz",
    "train_18561_a_2.nii.gz",
    "train_2534_a_1.nii.gz",
    "train_12684_a_1.nii.gz",
    "train_1742_b_1.nii.gz",
    "train_6829_a_2.nii.gz",
    "train_12543_a_2.nii.gz",
    "train_15604_a_2.nii.gz",
    "train_8936_a_2.nii.gz",
    "train_17413_a_1.nii.gz",
    "train_13416_a_1.nii.gz",
    "train_19971_a_1.nii.gz",
    "train_18744_a_1.nii.gz",
    "train_8080_b_1.nii.gz",
    "train_13785_a_1.nii.gz",
    "train_227_a_2.nii.gz",
    "train_3010_a_1.nii.gz",
    "train_12428_a_1.nii.gz",
    "train_18252_a_1.nii.gz",
    "train_15178_a_2.nii.gz",
    "train_12345_a_1.nii.gz",
    "train_7171_a_1.nii.gz",
    "train_17912_a_1.nii.gz",
    "train_17322_b_1.nii.gz",
    "train_13339_a_1.nii.gz",
    "train_17619_e_1.nii.gz",
    "train_11140_a_2.nii.gz",
    "train_13436_e_2.nii.gz",
    "train_7578_b_1.nii.gz",
    "train_3326_b_1.nii.gz",
    "train_14959_a_2.nii.gz",
    "train_9113_j_1.nii.gz",
    "train_5463_a_1.nii.gz",
    "train_10503_a_1.nii.gz",
    "train_8867_a_2.nii.gz",
    "train_3338_a_1.nii.gz",
    "train_12060_a_2.nii.gz",
    "train_13354_a_1.nii.gz",
    "train_4754_a_1.nii.gz",
    "train_4197_a_1.nii.gz",
    "train_8483_a_2.nii.gz",
    "train_12573_a_5.nii.gz",
    "train_1180_a_2.nii.gz",
    "train_9833_a_1.nii.gz",
    "train_12524_a_1.nii.gz",
    "train_15361_a_1.nii.gz",
    "train_19726_a_2.nii.gz",
    "train_17968_a_1.nii.gz",
    "train_19597_a_2.nii.gz",
    "train_5771_a_1.nii.gz",
    "train_2149_a_1.nii.gz",
    "train_3886_a_1.nii.gz",
    "train_7719_b_1.nii.gz",
    "train_2269_a_2.nii.gz",
    "train_5424_a_1.nii.gz",
    "train_2874_a_2.nii.gz",
    "train_13760_a_2.nii.gz",
    "train_19421_a_1.nii.gz",
    "train_1689_b_1.nii.gz",
    "train_17258_a_1.nii.gz",
    "train_12425_b_2.nii.gz",
    "train_14868_a_2.nii.gz",
    "train_4266_b_1.nii.gz",
    "train_1348_f_1.nii.gz",
    "train_17802_a_2.nii.gz",
    "train_13576_a_1.nii.gz",
    "train_4537_b_1.nii.gz",
    "train_1462_b_2.nii.gz",
    "train_3675_c_1.nii.gz",
    "train_12778_c_2.nii.gz",
    "train_11001_b_2.nii.gz",
    "train_5378_a_1.nii.gz",
    "train_6729_a_1.nii.gz",
    "train_10267_a_2.nii.gz",
    "train_1976_a_1.nii.gz",
    "train_4394_a_1.nii.gz",
    "train_19256_a_1.nii.gz",
    "train_2429_a_1.nii.gz",
    "train_11267_a_2.nii.gz",
    "train_11896_a_1.nii.gz",
    "train_8654_a_2.nii.gz",
    "train_3863_a_1.nii.gz",
    "train_12000_a_1.nii.gz",
    "train_7354_a_1.nii.gz",
    "train_7874_a_1.nii.gz",
    "train_12287_a_1.nii.gz",
    "train_5098_a_1.nii.gz",
    "train_12811_a_1.nii.gz",
    "train_15236_a_1.nii.gz",
    "train_18452_c_1.nii.gz",
    "train_8075_a_1.nii.gz",
    "train_8344_a_1.nii.gz",
    "train_14010_a_1.nii.gz",
    "train_8396_a_1.nii.gz",
    "train_9831_a_1.nii.gz",
    "train_488_a_1.nii.gz",
    "train_10097_a_1.nii.gz",
    "train_14587_a_1.nii.gz",
    "train_4488_a_1.nii.gz",
    "train_12928_a_1.nii.gz",
    "train_9611_a_1.nii.gz",
    "train_16841_a_1.nii.gz",
    "train_17729_a_2.nii.gz",
    "train_5410_a_2.nii.gz",
    "train_389_a_2.nii.gz",
    "train_6162_a_2.nii.gz",
    "train_19974_a_1.nii.gz",
    "train_16207_a_2.nii.gz",
    "train_11585_a_2.nii.gz",
    "train_6765_a_1.nii.gz",
    "train_12315_a_1.nii.gz",
    "train_5972_a_2.nii.gz",
    "train_14075_a_1.nii.gz",
    "train_7080_a_2.nii.gz",
    "train_17867_a_3.nii.gz",
    "train_12972_a_2.nii.gz",
    "train_11755_a_1.nii.gz",
    "train_11885_a_1.nii.gz",
    "train_1458_a_1.nii.gz",
    "train_9407_b_2.nii.gz",
    "train_18439_a_1.nii.gz",
    "train_7412_a_2.nii.gz",
    "train_9455_b_1.nii.gz",
    "train_13595_a_2.nii.gz",
    "train_4061_a_2.nii.gz",
    "train_3510_a_2.nii.gz",
    "train_10666_a_2.nii.gz",
    "train_8006_a_1.nii.gz",
    "train_19301_a_1.nii.gz",
    "train_18458_a_2.nii.gz",
    "train_16717_a_1.nii.gz",
    "train_16179_a_1.nii.gz",
    "train_9593_a_1.nii.gz",
    "train_7170_a_1.nii.gz",
    "train_11356_a_2.nii.gz",
    "train_5243_a_1.nii.gz",
    "train_3433_a_1.nii.gz",
    "train_2640_a_2.nii.gz",
    "train_11811_a_1.nii.gz",
    "train_4969_a_1.nii.gz",
    "train_15372_a_1.nii.gz",
    "train_7592_c_2.nii.gz",
    "train_10890_a_2.nii.gz",
    "train_7115_a_2.nii.gz",
    "train_6394_a_1.nii.gz",
    "train_3294_a_2.nii.gz",
    "train_14994_a_1.nii.gz",
    "train_18325_a_2.nii.gz",
    "train_17344_a_2.nii.gz",
    "train_7584_c_2.nii.gz",
    "train_11119_b_1.nii.gz",
    "train_9542_a_2.nii.gz",
    "train_13437_c_2.nii.gz",
    "train_4154_a_2.nii.gz",
    "train_12923_b_2.nii.gz",
    "train_11966_a_1.nii.gz",
    "train_9361_a_2.nii.gz",
    "train_13767_a_2.nii.gz",
    "train_18454_a_2.nii.gz",
    "train_16440_b_2.nii.gz",
    "train_1882_a_1.nii.gz",
    "train_1004_b_2.nii.gz",
    "train_13963_a_1.nii.gz",
    "train_8041_a_2.nii.gz",
    "train_17190_a_1.nii.gz",
    "train_3165_a_1.nii.gz",
    "train_15939_a_1.nii.gz",
    "train_9668_a_2.nii.gz",
    "train_6959_a_1.nii.gz",
    "train_11388_a_1.nii.gz",
    "train_71_a_2.nii.gz",
    "train_13297_a_1.nii.gz",
    "train_16955_a_2.nii.gz",
    "train_16012_a_1.nii.gz",
    "train_1754_a_1.nii.gz",
    "train_8583_a_2.nii.gz",
    "train_1415_a_2.nii.gz",
    "train_5557_a_2.nii.gz",
    "train_2248_a_2.nii.gz",
    "train_16873_a_1.nii.gz",
    "train_15143_b_2.nii.gz",
    "train_10439_a_2.nii.gz",
    "train_3025_a_2.nii.gz",
    "train_13812_a_1.nii.gz",
    "train_4832_a_1.nii.gz",
    "train_1184_b_1.nii.gz",
    "train_8063_a_1.nii.gz",
    "train_8848_a_2.nii.gz",
    "train_5786_b_2.nii.gz",
    "train_11152_b_1.nii.gz",
    "train_2426_a_2.nii.gz",
    "train_1488_a_2.nii.gz",
    "train_8782_a_5.nii.gz",
    "train_9204_a_1.nii.gz",
    "train_9829_a_3.nii.gz",
    "train_19350_a_1.nii.gz",
    "train_15907_a_1.nii.gz",
    "train_15438_a_1.nii.gz",
    "train_13955_a_2.nii.gz",
    "train_7702_b_2.nii.gz",
    "train_7173_b_1.nii.gz",
    "train_12337_a_2.nii.gz",
    "train_19427_b_1.nii.gz",
    "train_16830_a_2.nii.gz",
    "train_5404_b_2.nii.gz",
    "train_14115_a_2.nii.gz",
    "train_8629_a_2.nii.gz",
    "train_13188_a_1.nii.gz",
    "train_19922_a_1.nii.gz",
    "train_19164_b_2.nii.gz",
    "train_9997_a_2.nii.gz",
    "train_15839_a_1.nii.gz",
    "train_2280_a_2.nii.gz",
    "train_129_a_2.nii.gz",
    "train_14199_b_1.nii.gz",
    "train_3012_a_1.nii.gz",
    "train_1988_a_1.nii.gz",
    "train_15822_a_1.nii.gz",
    "train_16867_a_2.nii.gz",
    "train_724_a_2.nii.gz",
    "train_17791_c_2.nii.gz",
    "train_17429_a_2.nii.gz",
    "train_11530_b_1.nii.gz",
    "train_18091_a_2.nii.gz",
    "train_18284_a_2.nii.gz",
    "train_4129_a_1.nii.gz",
    "train_2572_a_1.nii.gz",
    "train_9789_a_2.nii.gz",
    "train_4200_a_2.nii.gz",
    "train_2781_a_1.nii.gz",
    "train_16810_a_2.nii.gz",
    "train_5117_c_1.nii.gz",
    "train_11480_a_1.nii.gz",
    "train_5832_a_1.nii.gz",
    "train_10085_a_1.nii.gz",
    "train_1857_a_1.nii.gz",
    "train_11762_a_1.nii.gz",
    "train_1592_a_1.nii.gz",
    "train_16904_a_2.nii.gz",
    "train_12332_a_2.nii.gz",
    "train_12160_c_1.nii.gz",
    "train_13335_b_2.nii.gz",
    "train_7239_a_1.nii.gz",
    "train_18313_b_1.nii.gz",
    "train_10774_a_1.nii.gz",
    "train_15328_a_1.nii.gz",
    "train_8017_a_2.nii.gz",
    "train_6537_a_1.nii.gz",
    "train_8695_a_1.nii.gz",
    "train_12262_a_1.nii.gz",
    "train_14726_a_1.nii.gz",
    "train_17300_a_1.nii.gz",
    "train_12845_a_1.nii.gz",
    "train_15508_a_2.nii.gz",
    "train_924_a_2.nii.gz",
    "train_7079_a_2.nii.gz",
    "train_5989_a_1.nii.gz",
    "train_1010_a_2.nii.gz",
    "train_8045_d_2.nii.gz",
    "train_1058_a_2.nii.gz",
    "train_16629_a_1.nii.gz",
    "train_12017_a_2.nii.gz",
    "train_19857_a_1.nii.gz",
    "train_6505_a_1.nii.gz",
    "train_8871_a_1.nii.gz",
    "train_12439_a_1.nii.gz",
    "train_14668_a_1.nii.gz",
    "train_4854_a_2.nii.gz",
    "train_6435_a_2.nii.gz",
    "train_3538_a_1.nii.gz",
    "train_8818_a_1.nii.gz",
    "train_10375_a_1.nii.gz",
    "train_8187_a_2.nii.gz",
    "train_5320_a_2.nii.gz",
    "train_18457_a_1.nii.gz",
    "train_11580_a_2.nii.gz",
    "train_14055_a_2.nii.gz",
    "train_10321_f_1.nii.gz",
    "train_5334_a_1.nii.gz",
    "train_15523_a_1.nii.gz",
    "train_16158_a_2.nii.gz",
    "train_807_a_1.nii.gz",
    "train_8774_a_2.nii.gz",
    "train_16164_a_2.nii.gz",
    "train_18793_b_2.nii.gz",
    "train_15602_a_1.nii.gz",
    "train_11626_a_1.nii.gz",
    "train_19795_a_1.nii.gz",
    "train_3574_a_2.nii.gz",
    "train_15230_a_1.nii.gz",
    "train_19926_a_2.nii.gz",
    "train_3711_a_2.nii.gz",
    "train_19306_a_2.nii.gz",
    "train_15717_a_2.nii.gz",
    "train_9454_a_2.nii.gz",
    "train_2085_a_1.nii.gz",
    "train_8004_a_1.nii.gz",
    "train_5150_a_2.nii.gz",
    "train_10575_a_1.nii.gz",
    "train_1422_c_1.nii.gz",
    "train_13067_a_3.nii.gz",
    "train_7540_e_1.nii.gz",
    "train_3748_b_1.nii.gz",
    "train_8344_a_2.nii.gz",
    "train_17732_e_2.nii.gz",
    "train_7811_b_1.nii.gz",
    "train_116_a_2.nii.gz",
    "train_2329_f_1.nii.gz",
    "train_19103_a_2.nii.gz",
    "train_8132_b_2.nii.gz",
    "train_5632_b_2.nii.gz",
    "train_11656_b_1.nii.gz",
    "train_7124_a_1.nii.gz",
    "train_1758_c_1.nii.gz",
    "train_3934_a_1.nii.gz",
    "train_727_a_2.nii.gz",
    "train_15667_a_1.nii.gz",
    "train_16642_a_2.nii.gz",
    "train_7975_a_1.nii.gz",
    "train_2374_a_1.nii.gz",
    "train_14164_a_2.nii.gz",
    "train_4561_a_2.nii.gz",
    "train_8583_a_1.nii.gz",
    "train_11221_a_2.nii.gz",
    "train_7033_a_2.nii.gz",
    "train_1914_a_2.nii.gz",
    "train_344_d_1.nii.gz",
    "train_4982_a_3.nii.gz",
    "train_19519_a_1.nii.gz",
    "train_3021_a_1.nii.gz",
    "train_10112_b_1.nii.gz",
    "train_12095_b_1.nii.gz",
    "train_12132_a_2.nii.gz",
    "train_4801_a_1.nii.gz",
    "train_15128_a_2.nii.gz",
    "train_18320_a_1.nii.gz",
    "train_882_a_2.nii.gz",
    "train_13191_a_1.nii.gz",
    "train_13638_a_2.nii.gz",
    "train_12005_a_2.nii.gz",
    "train_3375_a_2.nii.gz",
    "train_5963_a_1.nii.gz",
    "train_12364_a_1.nii.gz",
    "train_2139_d_1.nii.gz",
    "train_9110_a_2.nii.gz",
    "train_2093_a_1.nii.gz",
    "train_5419_a_2.nii.gz",
    "train_17942_a_1.nii.gz",
    "train_2558_a_2.nii.gz",
    "train_13905_a_2.nii.gz",
    "train_4089_a_1.nii.gz",
    "train_2511_a_1.nii.gz",
    "train_10875_a_2.nii.gz",
    "train_8773_a_2.nii.gz",
    "train_6850_a_1.nii.gz",
    "train_3186_b_1.nii.gz",
    "train_318_a_1.nii.gz",
    "train_7798_a_2.nii.gz",
    "train_11625_a_1.nii.gz",
    "train_19208_a_1.nii.gz",
    "train_14481_a_2.nii.gz",
    "train_4064_a_1.nii.gz",
    "train_12570_a_2.nii.gz",
    "train_7359_a_2.nii.gz",
    "train_15321_a_2.nii.gz",
    "train_13435_a_2.nii.gz",
    "train_8271_a_1.nii.gz",
    "train_5448_a_1.nii.gz",
    "train_19002_a_1.nii.gz",
    "train_4169_a_1.nii.gz"
]

# --- 6. Preprocessing and Feature Extraction ---
def load_and_preprocess_ct(path: str):
    transform = T.Compose([
        T.LoadImaged(keys=["image"]), T.EnsureChannelFirstd(keys=["image"]),
        T.ScaleIntensityRanged(keys=["image"], a_min=-1000, a_max=1000,
                               b_min=-1.0, b_max=1.0, clip=True),
        T.Resized(keys=["image"], spatial_size=(40, 480, 480),
                  mode="trilinear", align_corners=False),
        T.ToTensord(keys=["image"]),
    ])
    data = transform({"image": path})
    return data["image"].unsqueeze(0).to(DEVICE)

def reconstruct_path_from_filename(filename: str) -> str:
    parts = filename.replace('.nii.gz', '').split('_')
    train_id = parts[1]
    scan_id = parts[2]
    return f"dataset/train/train_{train_id}/train_{train_id}_{scan_id}/{filename}"

print("\n--- Extracting features for our mini-dataset ---")
training_data = []
for filename in sample_filenames:
    try:
        full_path = reconstruct_path_from_filename(filename)
        print(f"Processing {full_path}...")
        local_path = hf_hub_download(
            repo_id="ibrahimhamamci/CT-RATE",
            repo_type="dataset",
            filename=full_path,
            token=HF_TOKEN,
        )
        ct_tensor = load_and_preprocess_ct(local_path)

        with torch.no_grad():
            dummy_text_input = text_tokenizer_clip([""], return_tensors="pt", padding=True).to(DEVICE)
            _, image_embedding, _ = vision_model(dummy_text_input, ct_tensor, DEVICE, return_latents=True)

        labels_row = df_labels[df_labels['VolumeName'] == filename]
        if not labels_row.empty:
            label_vector = torch.tensor(labels_row.iloc[0, 1:].values.astype(np.float32)).to(DEVICE)
            training_data.append((image_embedding, label_vector))
    except Exception as e:
        print(f"Could not process file {filename}. Error: {e}. Skipping.")

print(f"\n ✅ Cell 2 complete. Created a mini-dataset with {len(training_data)} samples.")

# === 7. Save extracted features and labels for later reuse ===
os.makedirs("saved_artifacts", exist_ok=True)
image_embeddings = torch.cat([item[0] for item in training_data], dim=0)  # [N, 512]
label_vectors = torch.stack([item[1] for item in training_data], dim=0)   # [N, num_classes]

torch.save({
    "image_embeddings": image_embeddings,
    "label_vectors": label_vectors,
    "class_names": CLASS_NAMES,
}, "saved_artifacts/features_labels.pt")

# Save sample filename manifest
with open("saved_artifacts/sample_filenames.txt", "w") as f:
    for name in sample_filenames:
        full_path = reconstruct_path_from_filename(name)
        f.write(f"{name}\t{full_path}\n")

# Save a human-readable summary
summary_rows = []
for i, (emb, label) in enumerate(training_data):
    summary_rows.append({
        "sample_index": i,
        "volume_name": sample_filenames[i],
        "embedding_norm": emb.norm().item(),
        **{f"class_{j}": label[j].item() for j in range(label.shape[0])}
    })
df_summary = pd.DataFrame(summary_rows)
df_summary.to_csv("saved_artifacts/training_summary.csv", index=False)

print("✅ Saved features+labels, sample list, and summary to `saved_artifacts/`")


--- Using device: cuda ---

--- Loading CT-CLIP Model ---
 ✅ CT-CLIP model loaded and frozen.

--- Preparing labels ---
Found 18 classes.

--- Extracting features for our mini-dataset ---
Processing dataset/train/train_18373/train_18373_a/train_18373_a_2.nii.gz...


/home/user/miniconda/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


test all pooling
Processing dataset/train/train_19287/train_19287_a/train_19287_a_1.nii.gz...
test all pooling
Processing dataset/train/train_3282/train_3282_a/train_3282_a_1.nii.gz...
test all pooling
Processing dataset/train/train_54/train_54_a/train_54_a_2.nii.gz...
test all pooling
Processing dataset/train/train_12769/train_12769_a/train_12769_a_2.nii.gz...
test all pooling
Processing dataset/train/train_7066/train_7066_a/train_7066_a_1.nii.gz...
test all pooling
Processing dataset/train/train_2610/train_2610_a/train_2610_a_2.nii.gz...
test all pooling
Processing dataset/train/train_9742/train_9742_b/train_9742_b_2.nii.gz...
test all pooling
Processing dataset/train/train_11431/train_11431_a/train_11431_a_1.nii.gz...
test all pooling
Processing dataset/train/train_7613/train_7613_b/train_7613_b_1.nii.gz...
test all pooling
Processing dataset/train/train_644/train_644_d/train_644_d_1.nii.gz...
test all pooling
Processing dataset/train/train_13314/train_13314_a/train_13314_a_2.nii.gz

dataset/train/train_11280/train_11280_a/(…):   0%|          | 0.00/209M [00:00<?, ?B/s]

test all pooling
Processing dataset/train/train_11439/train_11439_a/train_11439_a_2.nii.gz...


...

dataset/train/train_15321/train_15321_a/(…):   0%|          | 0.00/171M [00:00<?, ?B/s]

test all pooling
Processing dataset/train/train_13435/train_13435_a/train_13435_a_2.nii.gz...


dataset/train/train_13435/train_13435_a/(…):   0%|          | 0.00/488M [00:00<?, ?B/s]

test all pooling
Processing dataset/train/train_8271/train_8271_a/train_8271_a_1.nii.gz...


dataset/train/train_8271/train_8271_a/tr(…):   0%|          | 0.00/86.7M [00:00<?, ?B/s]

In [12]:
# %%
# ==============================================================================
# CELL 3: Improved Classifier Head Training with K-Fold Cross-Validation
# ==============================================================================
import torch.nn as nn
import torch.optim as optim
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
import torch

# --- 1. Load the features and labels saved by Cell 2 ---
print("--- Loading pre-extracted features and labels ---")
# THE FIX: Add weights_only=False because we trust this file
saved_data = torch.load("saved_artifacts/features_labels.pt", weights_only=False)
all_embeddings = saved_data['image_embeddings']
all_labels = saved_data['label_vectors']
CLASS_NAMES = saved_data['class_names']
print(f"✅ Loaded {len(all_embeddings)} samples.")


# --- 2. Define a Deeper Classifier Head with Dropout ---
class DeeperClassifierHead(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate=0.5):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim)
        )
    def forward(self, x):
        return self.network(x)

# --- 3. K-Fold Cross-Validation Setup ---
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

print(f"\n--- Starting 5-Fold Cross-Validation Training on {len(all_embeddings)} samples ---")

for fold, (train_idx, val_idx) in enumerate(kf.split(all_embeddings)):
    print(f"\n--- Fold {fold+1}/5 ---")
    
    X_train, Y_train = all_embeddings[train_idx], all_labels[train_idx]
    X_val, Y_val = all_embeddings[val_idx], all_labels[val_idx]
    
    model = DeeperClassifierHead(
        input_dim=512, 
        hidden_dim=256, 
        output_dim=len(CLASS_NAMES)
    ).to(DEVICE)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # --- Training Loop ---
    num_epochs = 1000
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, Y_train)
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 20 == 0:
            print(f"  Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

    # --- Validation ---
    model.eval()
    with torch.no_grad():
        val_logits = model(X_val)
        val_probs = torch.sigmoid(val_logits)
        fold_results.append({
            "probs": val_probs.cpu().numpy(),
            "labels": Y_val.cpu().numpy()
        })

print("\n ✅ Cross-validation training complete.")

# --- 4. Aggregate Results and Tune Thresholds ---
all_val_probs = np.concatenate([res["probs"] for res in fold_results])
all_val_labels = np.concatenate([res["labels"] for res in fold_results])

best_thresholds = []
print("\n--- Tuning Thresholds on Cross-Validation Results ---")
for i in range(len(CLASS_NAMES)):
    best_f1 = 0
    best_thresh = 0.5
    for thresh in np.arange(0.1, 0.9, 0.05):
        preds = (all_val_probs[:, i] > thresh).astype(int)
        f1 = f1_score(all_val_labels[:, i], preds, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh
    best_thresholds.append(best_thresh)
    print(f"Best threshold for '{CLASS_NAMES[i]}': {best_thresh:.2f} (F1 Score: {best_f1:.3f})")


# --- 5. Train Final Model on ALL Data ---
print(f"\n--- Training final model on all {len(all_embeddings)} samples ---")
final_model = DeeperClassifierHead(
    input_dim=512, hidden_dim=256, output_dim=len(CLASS_NAMES)
).to(DEVICE)
optimizer = optim.Adam(final_model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

num_epochs_final = 100
for epoch in range(num_epochs_final):
    final_model.train()
    optimizer.zero_grad()
    outputs = final_model(all_embeddings)
    loss = criterion(outputs, all_labels)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 20 == 0:
        print(f"  Epoch [{epoch+1}/{num_epochs_final}], Loss: {loss.item():.4f}")

final_model.eval()
print("✅ Final model training complete.")

# --- 6. Save the final model and thresholds ---
torch.save({
    'model_state_dict': final_model.state_dict(),
    'thresholds': best_thresholds,
    'class_names': CLASS_NAMES
}, "final_classifier.pth")
print("\n✅ Final model and thresholds saved to 'final_classifier.pth'")

--- Loading pre-extracted features and labels ---
✅ Loaded 1000 samples.

--- Starting 5-Fold Cross-Validation Training on 1000 samples ---

--- Fold 1/5 ---
  Epoch [20/1000], Loss: 0.5152
  Epoch [40/1000], Loss: 0.4696
  Epoch [60/1000], Loss: 0.4636
  Epoch [80/1000], Loss: 0.4635
  Epoch [100/1000], Loss: 0.4629
  Epoch [120/1000], Loss: 0.4613
  Epoch [140/1000], Loss: 0.4609
  Epoch [160/1000], Loss: 0.4582
  Epoch [180/1000], Loss: 0.4578
  Epoch [200/1000], Loss: 0.4576
  Epoch [220/1000], Loss: 0.4550
  Epoch [240/1000], Loss: 0.4547
  Epoch [260/1000], Loss: 0.4522
  Epoch [280/1000], Loss: 0.4528
  Epoch [300/1000], Loss: 0.4518
  Epoch [320/1000], Loss: 0.4501
  Epoch [340/1000], Loss: 0.4493
  Epoch [360/1000], Loss: 0.4465
  Epoch [380/1000], Loss: 0.4452
  Epoch [400/1000], Loss: 0.4442
  Epoch [420/1000], Loss: 0.4416
  Epoch [440/1000], Loss: 0.4393
  Epoch [460/1000], Loss: 0.4389
  Epoch [480/1000], Loss: 0.4367
  Epoch [500/1000], Loss: 0.4355
  Epoch [520/1000], L

In [14]:
# %%
# ==============================================================================
# VERIFICATION CELL: Check the saved artifacts
# ==============================================================================
import torch
import os

print("--- Verifying saved files in 'saved_artifacts/' ---")

# 1. Define the path to the main data file
file_path = "saved_artifacts/features_labels.pt"

# 2. Check if the file exists and is not empty
if os.path.exists(file_path) and os.path.getsize(file_path) > 0:
    print(f"✅ File '{os.path.basename(file_path)}' exists and is not empty.")
    
    # 3. Load the data and check the number of samples
    # We use weights_only=False because we know the file is safe
    saved_data = torch.load(file_path, weights_only=False)
    
    image_embeddings = saved_data['image_embeddings']
    label_vectors = saved_data['label_vectors']
    
    num_samples = len(image_embeddings)
    print(f"✅ File contains data for {num_samples} samples.")
    
    # 4. Check the shape of the tensors
    print(f"   - Shape of image embeddings: {image_embeddings.shape}")
    print(f"   - Shape of label vectors: {label_vectors.shape}")
    
    if num_samples >= 1000:
        print("\nSUCCESS: The data for 1000+ samples has been saved correctly.")
    else:
        print("\nWARNING: The number of saved samples is less than 1000.")
        
else:
    print(f"❌ ERROR: File '{os.path.basename(file_path)}' was not found or is empty.")

--- Verifying saved files in 'saved_artifacts/' ---
✅ File 'features_labels.pt' exists and is not empty.
✅ File contains data for 1000 samples.
   - Shape of image embeddings: torch.Size([1000, 512])
   - Shape of label vectors: torch.Size([1000, 18])

SUCCESS: The data for 1000+ samples has been saved correctly.
